# Multi-tenant Chat with Papers - Query papers
## Get keys and urls

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

WEAVIATE_HTTP_URL = os.getenv("WEAVIATE_HTTP_URL")
WEAVIATE_GRPC_URL = os.getenv("WEAVIATE_GRPC_URL")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
USERNAME = os.getenv("USERNAME")
PAPERS_NAME = f"{USERNAME}_papers"

print(PAPERS_NAME)

## Connect to Weaviate

In [ ]:
import weaviate
# from weaviate.classes.init import AdditionalConfig, Timeout

client = weaviate.connect_to_custom(
    http_host=WEAVIATE_HTTP_URL,
    http_port=80,
    http_secure=False,
    grpc_host=WEAVIATE_GRPC_URL,
    grpc_port=50051,
    grpc_secure=False,

    headers = {
        "X-Azure-Api-Key": AZURE_OPENAI_API_KEY
    },

    # additional_config=AdditionalConfig(
    #     timeout=Timeout(init=2, query=45, insert=120),  # Values in seconds
    # )
)

client.is_ready()

## Vector search on tenants

In [ ]:
papers = client.collections.get(PAPERS_NAME)

ten = papers.with_tenant("2212-10496")

response = ten.query.near_text(
    query="Unsupervised learning",
    limit=5,
)

for item in response.objects:
    print(item.properties["chunk"])

## Genartive Search with tenants

In [ ]:
papers = client.collections.get(PAPERS_NAME)

ten2212 = papers.with_tenant("2212-10496")

response = ten2212.generate.near_text(
    query="Unsupervised learning",
    limit=5,
    single_prompt="What does the following text describe: {chunk}",
)

for item in response.objects:
    print(item.properties["chunk"])
    print(item.generative.text, '\n')

In [ ]:
papers = client.collections.get(PAPERS_NAME)

ten2212 = papers.with_tenant("2212-10496")

response = ten2212.generate.near_text(
    query="Unsupervised learning",
    limit=5,
    grouped_task="Explain how unsupervised learning works. Use only the provided content.",
    grouped_properties=["chunk"]
)

for item in response.objects:
    print(item.properties["chunk"])

print(response.generative.text)

In [ ]:
def paper_rag(paper_id, query, prompt):
    papers = client.collections.get(PAPERS_NAME)
    ten = papers.with_tenant(paper_id)

    response = ten.generate.near_text(
        query=query,
        limit=5,
        grouped_task=prompt + " Use only the provided content.",
        grouped_properties=["chunk"],
    )

    return {
        "title": response.objects[0].properties["title"],
        "source": [p.properties["chunk"] for p in response.objects],
        "generated": response.generative.text
    }

In [ ]:
paper_rag(
    "2212-10496",
    "Unsupervised learning",
    "Explain how unsupervised learning works"
)

In [ ]:
papers = client.collections.get(PAPERS_NAME)
papers.tenants.get()

## Close the client

In [ ]:
client.close()